# Lesson 3: Reflection and Blogpost Writing

## Setup

In [13]:
# llm_config = {"model": "gpt-3.5-turbo"}

llm_config = {
    "config_list": [
        {
            "model": "llama3.1",  # 
            "api_key": "ollama",  # 
            "base_url": "http://localhost:11434/v1",  
        }
    ],
    "cache_seed": None,  # Turns off caching, useful for testing different models
}

## The task!

In [14]:
task = '''
       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [15]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [16]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [17]:
import pprint
pprint.pprint(reply)

('**Unlocking AI Potential with DeepLearning.AI**\n'
 '\n'
 'DeepLearning.AI is an online platform that empowers beginners and experts '
 'alike to delve into the world of Artificial Intelligence (AI) and Machine '
 "Learning (ML). Founded by Google's Andrew Ng, this initiative aims to bridge "
 'the gap between industry professionals and those new to the field. Through '
 'lectures, tutorials, and hands-on projects, users can acquire in-demand '
 'skills in deep learning, neural networks, and more. By harnessing '
 "DeepLearning.AI's resources, learners can transform their careers, stay "
 "ahead of market demands, or simply cultivate a deeper understanding of AI's "
 'transformative power.')


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [18]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [19]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking the Power of AI with DeepLearning.AI**

DeepLearning.AI, a platform by Coursera and Andrew Ng, offers an extensive curriculum in machine learning and deep learning. With interactive lectures and hands-on exercises, learners can gain practical expertise in AI. The course's four-quadrant framework helps students understand the various applications and challenges of AI. Standouts include the Machine Learning Crash Course and Deep Learning Specialization. What sets this platform apart is its focus on real-world implications and the community-driven environment where learners collaborate, share insights, and get industry feedback. Upgrading your career with cutting-edge AI skills has never been easier!

---------------------

In [23]:
import pprint
pprint.pprint(res.summary)

('**Unlocking the Power of AI with DeepLearning.AI**\n'
 '\n'
 'DeepLearning.AI, a platform by Coursera and Andrew Ng, offers a '
 'comprehensive curriculum in machine learning and deep learning. With its '
 'interactive lectures and hands-on exercises, learners can gain practical '
 "expertise in AI. The course's four-quadrant framework helps students "
 'understand the various applications and challenges of AI.\n'
 '\n'
 'In particular, the **Machine Learning Crash Course** stands out for its '
 'clear and concise introduction to ML concepts, while the **Deep Learning '
 'Specialization** is praised for its structured approach to building complex '
 'neural networks. What sets this platform apart is its focus on real-world '
 'implications, as learners are encouraged to apply theoretical knowledge to '
 'practical problems.\n'
 '\n'
 'The community-driven environment where learners collaborate, share insights, '
 'and get industry feedback is also a significant highlight. This peer-t

In [38]:
print(res.chat_history[-1]['content'])

**Unlocking the Power of AI with DeepLearning.AI**

DeepLearning.AI, a platform by Coursera and Andrew Ng, offers a comprehensive curriculum in machine learning and deep learning. With its interactive lectures and hands-on exercises, learners can gain practical expertise in AI. The course's four-quadrant framework helps students understand the various applications and challenges of AI.

In particular, the **Machine Learning Crash Course** stands out for its clear and concise introduction to ML concepts, while the **Deep Learning Specialization** is praised for its structured approach to building complex neural networks. What sets this platform apart is its focus on real-world implications, as learners are encouraged to apply theoretical knowledge to practical problems.

The community-driven environment where learners collaborate, share insights, and get industry feedback is also a significant highlight. This peer-to-peer learning experience fosters growth, creativity, and critical thin

## Nested chat

In [39]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [40]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [41]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [45]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [46]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [47]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [48]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential with Deep Learning**

DeepLearning.AI, an initiative by Andrew Ng and colleagues, revolutionizes the way we approach Artificial Intelligence (AI) learning. This comprehensive platform offers a wide range of educational resources, from tutorials to courses, designed to equip learners with in-demand skills.

Through interactive lectures, hands-on exercises, and case studies, DeepLearning.AI provides an immersive experience that bridges the gap between theory and practice. The curriculum caters to beginners and seasoned professionals alike, ensuring everyone can grasp the power of deep learning with ease. Join this AI journey today!

-----------------------------------------------------------------------------

## Get the summary

In [15]:
print(res.summary)

Title: Mastering AI: Your Gateway to DeepLearning.AI

Embark on a transformative journey in artificial intelligence with DeepLearning.AI, a cutting-edge platform reshaping the industry. Curated for all skill levels, from beginners to experts, DeepLearning.AI offers comprehensive courses empowering you with in-depth knowledge of neural networks, deep learning, and more. Elevate your career and skills with expert guidance and hands-on experience. Join the thriving DeepLearning.AI community today and unlock endless opportunities in the AI landscape.

Don't miss out – dive into the world of AI with DeepLearning.AI now!

Incorporate the feedback received from reviewers and deliver a refined version of the blogpost on DeepLearning.AI.
